In [1]:
import tensorflow as tf
#@Mohamed Fathallah

In [2]:
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())
assert tf.test.is_gpu_available()
assert tf.test.is_built_with_cuda()
#Make sure your code run on GPU after you build a enviornment for it

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [3]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten
from tensorflow.keras.layers import BatchNormalization
#from tensorflow.keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras.layers import LeakyReLU
import matplotlib.pyplot as plt
import numpy as np

In [4]:
# The difference between the ReLU and the LeakyReLU is the ability of the latter to retain some degree 
# of the negative values that flow into it, whilst the former simply sets all values less than 0 to be 0
# . In theory, this extended output range offers a slightly higher flexibility to the model using it. 
# I'm sure the inventors thought it to be useful and perhaps proved that to be the case for a few benchmarks. 
# In practice, however, people generally just stick to the ReLU, 
# as the benefits of the LeakyReLU are not consistent and the ReLU is cheaper to compute 
# and therefore models train slightly faster.
#https://datascience.stackexchange.com/questions/39042/how-to-use-leakyrelu-as-activation-function-in-sequence-dnn-in-keraswhen-it-per

In [5]:
tf.keras.__version__

'2.9.0'

### Follow Code for Fixing some issue with GPU memory Growth

In [6]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [7]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    #https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth
    print(e)
    #Expected output physical device cannot be modified after being initialized

Physical devices cannot be modified after being initialized


## Starting The GAN Code

In [8]:
img_row=28
img_cols=28
channels=1
img_shape=(img_row,img_cols,channels)

In [165]:
#@This  i will change to use CNN
def build_generator():
    ##we can use Encode to give this laten later ()
    noise_shape=(100,) #1D array of size 100 (laten variable) 
    #i will try to build it using Conv Layer instead of dense Layer but first Test with Dense
    #or Use Transfer Learning and ResNet or Vgg
    model=Sequential()
    [
        Input(shape=noise_shape), #extra_line but for secure code
        #Dense(512,activation=activations.relu) #beacuse i want to add Leakyrelu
        Dense(256), #this layer will use the next layer activation function it's ,if you didn't pass any activation it will be linear model as the activation add the non-linearity
        LeakyReLU(alpha=0.2),
        BatchNormalization(),
        #Batch-norm does help to  normalize output from the activation function  increase speed of  traning , 
        #you have normzlie data come in and each layer also 
        #L2
        Dense(512),
        LeakyReLU(alpha=0.2),
        BatchNormalization(),
        #L3
        Dense(1024),
        LeakyReLU(alpha=0.2),
        BatchNormalization(),
        #Output Layer
        Dense(units=np.prod(img_shape),activation="tanh"), #output each pixel from neural 
        Reshape(img_shape) #Layer that reshapes inputs into the given shape. #inputshape =taken from pevious layer which is the same as img_shape no problem will occur
        #Reshape will output the dense layer into shape of image dim
    ]
    noise=Input(shape=noise_shape)
    img=model(noise) # create model with input shape and it will gives you the output shape
    
    return Model(noise,img) #takes inputs and outputs and build the model
    
    #this combination of functional and sequentional API
    
#https://keras.io/api/layers/reshaping_layers/reshape/

#This take laten vector and output image shape Takes (100) ,output (28,28,1)

In [166]:
def build_discriminator():
    
    model=Sequential()
    [
        #@ Recommnded to pass the input_shape=image_shape ,but it can do it on it's own
        Input(shape=img_shape), #this mean it's takes (28,28,3)
        Flatten(), # this will convert the image from 3d or any thing to 1D Density layer 
        Dense(512),
        LeakyReLU(alpha=0.2),
        Dense(256),
        LeakyReLU(alpha=0.2),
        Dense(1,activation="sigmoid") #True or False # number image or not
        #Just 2 Layers
    ]
    
    img=Input(shape=img_shape) #Takes image then Flatten it  you can do it by reshape the image data to (-1,28,28) 
    #then input shape will be (28*28) (Flatten() solve this for you)
    validity=model(img)
    return Model(img,validity)
#This gives you this is real image or fake image

# Very Important part of train

In [167]:
(x_train,_),(_,_)=mnist.load_data()

In [168]:
x_train=x_train.astype("float32")/255.0 #Scale data 0 to 1

In [169]:
x_train.shape  #i want the shape be (28,28,1) gray scale images

(60000, 28, 28)

In [170]:
x_train=np.expand_dims(x_train,axis=3)

In [171]:
x_train.shape

(60000, 28, 28, 1)

In [11]:
def train(epochs, batch_size=128, save_interval=50):
    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Convert to float and Rescale -1 to 1 (Can also do 0 to 1)
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5

    #Add channels dimension. As the input to our gen and discr. has a shape 28x28x1.
    X_train = np.expand_dims(X_train, axis=3) 
    
    #Want to have of batch to be from fake image and another half from real images  (to make the batch of discriminator)
    half_batch = int(batch_size / 2)
    
    #1-We have to Train discriminator  then generator and so on
    #We then loop through a number of epochs to train our Discriminator by first selecting
    #a random batch of images from our true dataset, generating a set of images from our
    #Generator, feeding both set of images into our Discriminator, and finally setting the
    #loss parameters for both the real and fake images, as well as the combined loss. 
    
    for epoch in range(epochs):
        
        # Select a random half batch of real images
        idx = np.random.randint(0, X_train.shape[0], half_batch) #from 0 to 60000 select 64 number if batch_size=128
        imgs = X_train[idx] #slice by list of number    #@@@@@@@@@@
        #shape (64,28,28,1)
        
        #@ this laten can be changed to be autoencoder of 64 image it currently values from 0 to 1 (64,100)
        noise = np.random.normal(0, 1, (half_batch, 100)) 
        #shape(64,100)
        #discriminator takes shape (28,28,1)
        #so those (64,100) have to passed to generator to give me output of 64 image of (28,28,1)
        gen_imgs=generator.predict(noise)                #@@@@@@@@@@@
        #shape (64,28,28,1)
        
        #Train on batch of real image and batch of fake images (discriminator)
        d_loss_real=discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1))) #x (64,28,28,1) y=(64,1) for the 64 image
        
        #Take the average loss  from real and fake image  (so far i calculate the loss of  discrimintor for 2 batches)
        d_loss=0.5*np.add(d_loss_real,d_loss_fake)
        #combine the loss normally as the los function of entropy loss function (this loss is the loss of discriminator)
        
        #(سهل تدخلو الحقيقى و المزيف و تقوله و تطلع منو اللوسس)
        #Now Train Generator and do nothing to the discriminator
        #Done with discriminator within the same epoch go ahead and train the generator
        #اعمل تدريب للجنريتو  بحيث  الدسرمنتور يخلى الداتا بتعتو فلد ب1
        #And within the same loop we train our Generator, by setting the input noise and
        #ultimately training the Generator to have the Discriminator label its samples as valid
        #by specifying the gradient loss.
        
        noise = np.random.normal(0, 1, (batch_size, 100)) #generate noise again but 128 sample now
        #Generate corresponding truth values
        # The generator wants the discriminator to label the generated samples
        # as valid (ones)
        #This is where the genrator is trying to trick discriminator into believing
        #the generated image is true (hence value of 1 for y)
        #(الداتا الهعملها هديها للدسكرمنتور واقوله دى ترو خلي بالك)
        valid_y = np.array([1] * batch_size) #دى كده هتكون الواى بتاعة الداتا الهعملها بالجنرتور دلوقتي
        
        #Generator هيدرب بانو هيكون ضمن موديل بيكون الداخل ليه الصوره العملتها الجنرتور و الطالع منو النتيجه بتاعة الصوره ده من الدسكرمنتور
        
        g_loss = combined.train_on_batch(noise, valid_y) #Noise will be z  
        
        #combined model takes the nosise as input and the output is result from discriminator the data i give is (fake,1) and take the loss from discriminator and start modify generator
        
        #make the noise and generate the image from this noise pass the img to discriminator take the loss function 
        #build new Model takes (z the laten vector, discriminator(img)) where img  generated from generator so this model will say
        #this laten discriminaotr gives this result   0 most of time 
        #combined method takes (z,validity) and train generator based on the result of discriminator for it's input
        
        
        

In [12]:
#valid_y = np.array([1] * 128)

In [13]:
(X_train, _), (_, _) = mnist.load_data()
X_train.shape

(60000, 28, 28)

In [1]:
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("C:\\Users\\legion\\OneDrive - fci.kfs.edu.eg\\Desktop\\Unet\mnist_%d.png" % epoch)
    plt.close()
#This function saves our images for us to view|

In [193]:
def build_generator():

    noise_shape = (100,) #1D array of size 100 (latent vector / noise)

#Define your generator network 
#Here we are only using Dense layers. But network can be complicated based
#on the application. For example, you can use VGG for super res. GAN.         

    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)    #Generated image

    return Model(noise, img)

#Alpha — α is a hyperparameter which controls the underlying value to which the
#function saturates negatives network inputs.
#Momentum — Speed up the training
##########################################################################

#Given an input image, the Discriminator outputs the likelihood of the image being real.
    #Binary classification - true or false (we're calling it validity)

def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)
#The validity is the Discriminator’s guess of input being real or not.


#Now that we have constructed our two models it’s time to pit them against each other.
#We do this by defining a training function, loading the data set, re-scaling our training
#images and setting the ground truths. 
def train(epochs, batch_size=128, save_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Convert to float and Rescale -1 to 1 (Can also do 0 to 1)
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5

#Add channels dimension. As the input to our gen and discr. has a shape 28x28x1.
    X_train = np.expand_dims(X_train, axis=3) 

    half_batch = int(batch_size / 2)


#We then loop through a number of epochs to train our Discriminator by first selecting
#a random batch of images from our true dataset, generating a set of images from our
#Generator, feeding both set of images into our Discriminator, and finally setting the
#loss parameters for both the real and fake images, as well as the combined loss. 
    
    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half batch of real images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]

 
        noise = np.random.normal(0, 1, (half_batch, 100))

        # Generate a half batch of fake images
        gen_imgs = generator.predict(noise)

        # Train the discriminator on real and fake images, separately
        #Research showed that separate training is more effective. 
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    #take average loss from real and fake images. 
    #
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 

#And within the same loop we train our Generator, by setting the input noise and
#ultimately training the Generator to have the Discriminator label its samples as valid
#by specifying the gradient loss.
        # ---------------------
        #  Train Generator
        # ---------------------
#Create noise vectors as input for generator. 
#Create as many noise vectors as defined by the batch size. 
#Based on normal distribution. Output will be of size (batch size, 100)
        noise = np.random.normal(0, 1, (batch_size, 100)) 

        # The generator wants the discriminator to label the generated samples
        # as valid (ones)
        #This is where the genrator is trying to trick discriminator into believing
        #the generated image is true (hence value of 1 for y)
        valid_y = np.array([1] * batch_size) #Creates an array of all ones of size=batch size

        # Generator is part of combined where it got directly linked with the discriminator
        # Train the generator with noise as x and 1 as y. 
        # Again, 1 as the output as it is adversarial and if generator did a great
        #job of folling the discriminator then the output would be 1 (true)
        g_loss = combined.train_on_batch(noise, valid_y)


#Additionally, in order for us to keep track of our training process, we print the
#progress and save the sample image output depending on the epoch interval specified.  
# Plot the progress
        
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if epoch % save_interval == 0:
            save_imgs(epoch)

#when the specific sample_interval is hit, we call the
#sample_image function. Which looks as follows.




##############################################################################

#Let us also define our optimizer for easy use later on.
#That way if you change your mind, you can change it easily here
optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.

# Build and compile the discriminator first. 
#Generator will be trained as part of the combined model, later. 
#pick the loss function and the type of metric to keep track.                 
#Binary cross entropy as we are doing prediction and it is a better
#loss function compared to MSE or other. 
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

#build and compile our Discriminator, pick the loss function

#SInce we are only generating (faking) images, let us not track any metrics.
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

##This builds the Generator and defines the input noise. 
#In a GAN the Generator network takes noise z as an input to produce its images.  
z = Input(shape=(100,))   #Our random input to the generator
img = generator(z)

#This ensures that when we combine our networks we only train the Generator.
#While generator training we do not want discriminator weights to be adjusted. 
#This Doesn't affect the above descriminator training.     
discriminator.trainable = False  

#This specifies that our Discriminator will take the images generated by our Generator
#and true dataset and set its output to a parameter called valid, which will indicate
#whether the input is real or not.  
valid = discriminator(img)  #Validity check on the generated image


#Here we combined the models and also set our loss function and optimizer. 
#Again, we are only training the generator here. 
#The ultimate goal here is for the Generator to fool the Discriminator.  
# The combined model  (stacked generator and discriminator) takes
# noise as input => generates images => determines validity

combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)


train(epochs=10000, batch_size=32, save_interval=1000)

#Save model for future use to generate fake images
#Not tested yet... make sure right model is being saved..
#Compare with GAN4

generator.save('generator_model.h5')  #Test the model on GAN4_predict...
#Change epochs back to 30K
                
#Epochs dictate the number of backward and forward propagations, the batch_size
#indicates the number of training samples per backward/forward propagation, and the
#sample_interval specifies after how many epochs we call our sample_image function.

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_11 (Flatten)        (None, 784)               0         
                                                                 
 dense_115 (Dense)           (None, 512)               401920    
                                                                 
 leaky_re_lu_82 (LeakyReLU)  (None, 512)               0         
                                                                 
 dense_116 (Dense)           (None, 256)               131328    
                                                                 
 leaky_re_lu_83 (LeakyReLU)  (None, 256)               0         
                                                                 
 dense_117 (Dense)           (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-t

1/1 [==============================] - 0s 12ms/step
45 [D loss: 0.022721, acc.: 100.00%] [G loss: 3.853344]
1/1 [==============================] - 0s 12ms/step
46 [D loss: 0.015033, acc.: 100.00%] [G loss: 3.789346]
1/1 [==============================] - 0s 14ms/step
47 [D loss: 0.013657, acc.: 100.00%] [G loss: 3.804997]
1/1 [==============================] - 0s 12ms/step
48 [D loss: 0.024962, acc.: 100.00%] [G loss: 3.759226]
1/1 [==============================] - 0s 12ms/step
49 [D loss: 0.018334, acc.: 100.00%] [G loss: 3.861562]
1/1 [==============================] - 0s 13ms/step
50 [D loss: 0.011052, acc.: 100.00%] [G loss: 4.125482]
1/1 [==============================] - 0s 13ms/step
51 [D loss: 0.017814, acc.: 100.00%] [G loss: 3.928224]
1/1 [==============================] - 0s 15ms/step
52 [D loss: 0.018389, acc.: 100.00%] [G loss: 3.899610]
1/1 [==============================] - 0s 13ms/step
53 [D loss: 0.017288, acc.: 100.00%] [G loss: 4.038988]
1/1 [=======================

1/1 [==============================] - 0s 13ms/step
121 [D loss: 0.049975, acc.: 100.00%] [G loss: 4.962688]
1/1 [==============================] - 0s 13ms/step
122 [D loss: 0.047053, acc.: 100.00%] [G loss: 6.427314]
1/1 [==============================] - 0s 13ms/step
123 [D loss: 0.603357, acc.: 84.38%] [G loss: 3.161118]
1/1 [==============================] - 0s 14ms/step
124 [D loss: 0.212213, acc.: 96.88%] [G loss: 3.365126]
1/1 [==============================] - 0s 13ms/step
125 [D loss: 0.123481, acc.: 93.75%] [G loss: 4.547642]
1/1 [==============================] - 0s 12ms/step
126 [D loss: 0.059293, acc.: 96.88%] [G loss: 5.759512]
1/1 [==============================] - 0s 13ms/step
127 [D loss: 0.003528, acc.: 100.00%] [G loss: 5.897285]
1/1 [==============================] - 0s 12ms/step
128 [D loss: 0.009314, acc.: 100.00%] [G loss: 5.737559]
1/1 [==============================] - 0s 12ms/step
129 [D loss: 0.035769, acc.: 100.00%] [G loss: 5.474727]
1/1 [==================

1/1 [==============================] - 0s 13ms/step
197 [D loss: 0.467552, acc.: 68.75%] [G loss: 4.030972]
1/1 [==============================] - 0s 14ms/step
198 [D loss: 0.248534, acc.: 90.62%] [G loss: 3.051541]
1/1 [==============================] - 0s 12ms/step
199 [D loss: 0.126755, acc.: 96.88%] [G loss: 3.697824]
1/1 [==============================] - 0s 13ms/step
200 [D loss: 0.561401, acc.: 68.75%] [G loss: 2.220885]
1/1 [==============================] - 0s 12ms/step
201 [D loss: 0.224948, acc.: 87.50%] [G loss: 3.273216]
1/1 [==============================] - 0s 13ms/step
202 [D loss: 0.141812, acc.: 96.88%] [G loss: 3.047460]
1/1 [==============================] - 0s 13ms/step
203 [D loss: 0.247367, acc.: 93.75%] [G loss: 2.711372]
1/1 [==============================] - 0s 13ms/step
204 [D loss: 0.409337, acc.: 78.12%] [G loss: 1.914833]
1/1 [==============================] - 0s 13ms/step
205 [D loss: 0.143843, acc.: 96.88%] [G loss: 3.153356]
1/1 [=======================

1/1 [==============================] - 0s 13ms/step
273 [D loss: 0.811931, acc.: 40.62%] [G loss: 0.697753]
1/1 [==============================] - 0s 12ms/step
274 [D loss: 0.563348, acc.: 59.38%] [G loss: 0.905870]
1/1 [==============================] - 0s 12ms/step
275 [D loss: 0.677807, acc.: 56.25%] [G loss: 0.801176]
1/1 [==============================] - 0s 13ms/step
276 [D loss: 0.596598, acc.: 62.50%] [G loss: 0.816981]
1/1 [==============================] - 0s 12ms/step
277 [D loss: 0.717600, acc.: 43.75%] [G loss: 0.695899]
1/1 [==============================] - 0s 13ms/step
278 [D loss: 0.756078, acc.: 40.62%] [G loss: 0.675532]
1/1 [==============================] - 0s 12ms/step
279 [D loss: 0.605029, acc.: 53.12%] [G loss: 0.918097]
1/1 [==============================] - 0s 12ms/step
280 [D loss: 0.871370, acc.: 21.88%] [G loss: 0.718278]
1/1 [==============================] - 0s 12ms/step
281 [D loss: 0.668755, acc.: 46.88%] [G loss: 0.718479]
1/1 [=======================

...

1/1 [==============================] - 0s 14ms/step
9357 [D loss: 0.778438, acc.: 43.75%] [G loss: 0.899065]
1/1 [==============================] - 0s 12ms/step
9358 [D loss: 0.619260, acc.: 71.88%] [G loss: 0.993693]
1/1 [==============================] - 0s 13ms/step
9359 [D loss: 0.723531, acc.: 56.25%] [G loss: 0.869512]
1/1 [==============================] - 0s 15ms/step
9360 [D loss: 0.613104, acc.: 62.50%] [G loss: 0.852760]
1/1 [==============================] - 0s 14ms/step
9361 [D loss: 0.657172, acc.: 62.50%] [G loss: 0.809950]
1/1 [==============================] - 0s 14ms/step
9362 [D loss: 0.702918, acc.: 59.38%] [G loss: 0.875798]
1/1 [==============================] - 0s 13ms/step
9363 [D loss: 0.690426, acc.: 46.88%] [G loss: 0.906654]
1/1 [==============================] - 0s 14ms/step
9364 [D loss: 0.669498, acc.: 59.38%] [G loss: 0.881522]
1/1 [==============================] - 0s 14ms/step
9365 [D loss: 0.722849, acc.: 46.88%] [G loss: 0.877044]
1/1 [==============

1/1 [==============================] - 0s 13ms/step
9507 [D loss: 0.714426, acc.: 46.88%] [G loss: 0.877798]
1/1 [==============================] - 0s 12ms/step
9508 [D loss: 0.608806, acc.: 71.88%] [G loss: 0.867593]
1/1 [==============================] - 0s 14ms/step
9509 [D loss: 0.700757, acc.: 43.75%] [G loss: 0.887782]
1/1 [==============================] - 0s 13ms/step
9510 [D loss: 0.633396, acc.: 65.62%] [G loss: 0.825309]
1/1 [==============================] - 0s 14ms/step
9511 [D loss: 0.667840, acc.: 65.62%] [G loss: 0.888130]
1/1 [==============================] - 0s 14ms/step
9512 [D loss: 0.693763, acc.: 68.75%] [G loss: 0.891246]
1/1 [==============================] - 0s 13ms/step
9513 [D loss: 0.589382, acc.: 62.50%] [G loss: 0.877848]
1/1 [==============================] - 0s 13ms/step
9514 [D loss: 0.709218, acc.: 46.88%] [G loss: 0.935444]
1/1 [==============================] - 0s 13ms/step
9515 [D loss: 0.672325, acc.: 56.25%] [G loss: 0.884440]
1/1 [==============

1/1 [==============================] - 0s 13ms/step
9657 [D loss: 0.627444, acc.: 50.00%] [G loss: 0.860109]
1/1 [==============================] - 0s 14ms/step
9658 [D loss: 0.699710, acc.: 53.12%] [G loss: 0.853348]
1/1 [==============================] - 0s 12ms/step
9659 [D loss: 0.781560, acc.: 46.88%] [G loss: 0.805419]
1/1 [==============================] - 0s 12ms/step
9660 [D loss: 0.685777, acc.: 56.25%] [G loss: 0.849786]
1/1 [==============================] - 0s 13ms/step
9661 [D loss: 0.667993, acc.: 59.38%] [G loss: 0.845035]
1/1 [==============================] - 0s 13ms/step
9662 [D loss: 0.699998, acc.: 59.38%] [G loss: 0.825149]
1/1 [==============================] - 0s 13ms/step
9663 [D loss: 0.622080, acc.: 62.50%] [G loss: 0.826807]
1/1 [==============================] - 0s 13ms/step
9664 [D loss: 0.647948, acc.: 65.62%] [G loss: 0.812737]
1/1 [==============================] - 0s 13ms/step
9665 [D loss: 0.664081, acc.: 59.38%] [G loss: 0.853712]
1/1 [==============

1/1 [==============================] - 0s 13ms/step
9807 [D loss: 0.646784, acc.: 62.50%] [G loss: 0.874307]
1/1 [==============================] - 0s 15ms/step
9808 [D loss: 0.663225, acc.: 56.25%] [G loss: 0.875365]
1/1 [==============================] - 0s 13ms/step
9809 [D loss: 0.724530, acc.: 56.25%] [G loss: 0.883556]
1/1 [==============================] - 0s 13ms/step
9810 [D loss: 0.650398, acc.: 56.25%] [G loss: 0.895746]
1/1 [==============================] - 0s 13ms/step
9811 [D loss: 0.704512, acc.: 46.88%] [G loss: 0.830686]
1/1 [==============================] - 0s 13ms/step
9812 [D loss: 0.642746, acc.: 65.62%] [G loss: 0.891793]
1/1 [==============================] - 0s 13ms/step
9813 [D loss: 0.657950, acc.: 65.62%] [G loss: 0.881612]
1/1 [==============================] - 0s 14ms/step
9814 [D loss: 0.704587, acc.: 56.25%] [G loss: 0.897393]
1/1 [==============================] - 0s 15ms/step
9815 [D loss: 0.551582, acc.: 78.12%] [G loss: 0.929248]
1/1 [==============

1/1 [==============================] - 0s 14ms/step
9957 [D loss: 0.652495, acc.: 62.50%] [G loss: 0.918121]
1/1 [==============================] - 0s 13ms/step
9958 [D loss: 0.762341, acc.: 46.88%] [G loss: 0.815818]
1/1 [==============================] - 0s 12ms/step
9959 [D loss: 0.640180, acc.: 65.62%] [G loss: 0.818601]
1/1 [==============================] - 0s 13ms/step
9960 [D loss: 0.584749, acc.: 75.00%] [G loss: 0.833831]
1/1 [==============================] - 0s 14ms/step
9961 [D loss: 0.708240, acc.: 56.25%] [G loss: 0.863642]
1/1 [==============================] - 0s 13ms/step
9962 [D loss: 0.717337, acc.: 65.62%] [G loss: 0.879017]
1/1 [==============================] - 0s 14ms/step
9963 [D loss: 0.628762, acc.: 65.62%] [G loss: 0.910936]
1/1 [==============================] - 0s 14ms/step
9964 [D loss: 0.691825, acc.: 53.12%] [G loss: 0.895979]
1/1 [==============================] - 0s 13ms/step
9965 [D loss: 0.664295, acc.: 53.12%] [G loss: 0.893990]
1/1 [==============

In [5]:
import matplotlib.pyplot as plt

1.0